<a href="https://colab.research.google.com/github/yukliio/stemcamp.aistation/blob/main/pneumonia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem statement

# Step 1: importing the Dataset

In [2]:
!pip install kagglehub

fatal: not a git repository (or any of the parent directories): .git
